#### Given a list of Git repositories, create containers and install dependencies for them - Python 3.11 recommended for conda env to run this script

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path=os.getcwd()+"/env")

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')

Extract github links from bookmarks

In [ ]:
from bs4 import BeautifulSoup

# Define the path to the bookmarks HTML file on the Desktop
bookmarks_file_name = 'bookmarks_6_3_25.html'
desktop_path = os.path.expanduser('~/Desktop')
bookmarks_file_path = os.path.join(desktop_path, bookmarks_file_name)

github_links = []

try:
    with open(bookmarks_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the Tutorials folder
    tutorials_folder = soup.find('h3', string='Tutorials')
    
    if tutorials_folder:
        # Get all links within the Tutorials folder's parent DT tag
        tutorial_links = tutorials_folder.find_parent('dt').find_all('a')
        
        # Filter for GitHub links
        for link_tag in tutorial_links:
            href = link_tag.get('href')
            if href and '/github.com' in href:
                # Split URL by '/' and keep only up to owner/repo
                parts = href.split('/')
                if len(parts) > 5:  # github.com/owner/repo would be 5 parts
                    cleaned_url = '/'.join(parts[:5])
                else:
                    cleaned_url = href
                github_links.append(cleaned_url)

        print(f"Found {len(github_links)} GitHub tutorial links:")
        for link in github_links:
            print(link)
    else:
        print("Tutorials folder not found in bookmarks")

except FileNotFoundError:
    print(f"Error: The file '{bookmarks_file_path}' was not found.")
    print("Please ensure the bookmarks file is on your Desktop and the name is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
github_links = list(dict.fromkeys(github_links))
print(f"{len(github_links)} unique links found")

github_links_subset=github_links[96:] #gpt4.1 costs ~14cents for 20 repos (83cents for ~100 repos), gpt4omini costs ~2cents

Run only git links in folder 'Tutorials' in bookmarks

In [ ]:
from bs4 import BeautifulSoup, Tag # Added Tag for type checking

def extract_links_from_tutorials(html_content):
    """
    Extracts links from the 'Tutorials' folder in a bookmarks HTML file,
    excluding links within subfolders of 'Tutorials'.

    Args:
        html_content (str): The HTML content of the bookmarks file.

    Returns:
        list: A list of dictionaries, where each dictionary contains
              the 'text' and 'href' of a bookmark.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    links = []

    # Find the H3 tag for the "Tutorials" folder
    tutorials_folder_h3 = None
    for h3_tag_candidate in soup.find_all('h3'):
        if h3_tag_candidate.get_text(strip=True).lower() == 'tutorials': # Case-insensitive match
            tutorials_folder_h3 = h3_tag_candidate
            break

    if not tutorials_folder_h3:
        print("Could not find a 'Tutorials' folder.")
        return links

    # The list of bookmarks and subfolders is usually in the next <DL> tag
    dl_tag = tutorials_folder_h3.find_next_sibling('dl')

    if not dl_tag:
        print("Could not find the <DL> list associated with the 'Tutorials' folder.")
        return links

    # Iterate through each direct child of the folder's <DL> tag.
    # A direct child might be a <P> tag (which then contains DTs/DLs),
    # or less commonly, <DT> tags directly.
    for direct_child_of_dl in dl_tag.children:
        if not isinstance(direct_child_of_dl, Tag):
            continue # Skip NavigableStrings like newlines

        items_to_scan_in_block = []
        if direct_child_of_dl.name == 'p':
            # If the direct child of <DL> is a <P>, its children are the actual items (DTs, DLs)
            items_to_scan_in_block = direct_child_of_dl.children
        elif direct_child_of_dl.name == 'dt':
            # If the direct child of <DL> is a <DT>, process it directly.
            items_to_scan_in_block = [direct_child_of_dl]
        # If direct_child_of_dl is another <DL> directly under the Tutorials <DL>
        # (e.g., a subfolder's content list not wrapped in a <DT><H3> first),
        # items_to_scan_in_block will remain empty, and the inner loop is skipped for it.
        # print(items_to_scan_in_block)
        for element in items_to_scan_in_block:
            if not isinstance(element, Tag):
                continue # Skip NavigableStrings
            # We are interested in <DT> elements at this level
            if element.name == 'dt':
                def extract_top_level_links_with_names(tag):
                    def walk(node, depth):
                        if isinstance(node, Tag):
                            if node.name == 'a' and depth == 0:
                                links.append((node.get_text(strip=True), node['href']))
                            new_depth = depth + 1 if node.name == 'dl' else depth
                            for child in node.children:
                                walk(child, new_depth)

                    links = []
                    walk(tag, 0)
                    return links

                links = extract_top_level_links_with_names(element)
                # print(links)

    return links

# --- Example Usage ---
# Load the HTML content from your file
file_path = 'bookmarks_6_3_25.html' # Make sure this is your file

try:
    with open(file_path, 'r', encoding='utf-8') as f: #
        html_doc = f.read() #

    # Extract the links
    tutorial_links = extract_links_from_tutorials(html_doc)

    # Print the extracted links
    if tutorial_links:
        print("\nLinks found directly in 'Tutorials' folder (excluding subfolders):")
        main_links = []
        main_names = []
        for name, link in tutorial_links:
            print(f"{name}: {link}")
            main_links.append(link)
            main_names.append(name)

    else:
        print("No links were found directly under the 'Tutorials' folder, or the folder itself was not found.")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print(f"Before filtering, {len(main_links)} links were found")
main_links = [item for item in main_links if 'github' in item]
print(f"After filtering, {len(main_links)} links containing 'github' in their URL remain.")

In [20]:
github_links_subset = main_links[:100]#[:len(main_links)//2]  # Get first half of the links

### Docker

Sequential run

Only for pip installs

In [ ]:
# import asyncio
# import json
# import openai, subprocess, os, re
# from gitingest import ingest
# import docker
# from pathlib import Path
# from tqdm import tqdm
# import time

# # Initialize Docker client
# docker_client = docker.from_env()

# # Global tracking dictionaries
# error_log = {}  # repo_name -> error_string
# requirements_successful = {}  # repo_name -> container_id
# code_successful = {}  # repo_name -> container_id
# requirements_successful_code_failed = {}  # repo_name -> container_id

# async def setup_repo(git_url, base_dir='repos'):
#     repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
#     print("Repo name is:", repo_name)
#     repo_dir = Path(base_dir) / repo_name
#     # print("Repo dir is:", repo_dir)
#     if not repo_dir.exists():
#         # Run blocking subprocess in a thread
#         await asyncio.to_thread(subprocess.run, ['git', 'clone', git_url, str(repo_dir)], check=True)
#     return repo_dir

# async def ingest_repo(repo_dir): # Made async
#     # Run blocking ingest call in a thread
#     # Changed include_patterns to be a set of strings as expected by gitingest
#     summary, tree, content = await asyncio.to_thread(ingest, str(repo_dir), include_patterns={"README.md", "requirements.txt"})
#     return summary, tree, content

# async def extract_main_code(documents): # Made async
#     prompt = (
#         "Given the following content from a GitHub repo, extract or synthesize a minimal executable .py script "
#         "that demonstrates how to use the package or run its main functionality."
#         "The output should be valid Python code for a `test_code.py` file. If there's nothing executable, just output "
#         "`print(\"no code found\")`. The content is: "
#         f"{documents} \n \n"
#     )
#     # Use async version of OpenAI call
#     response = await openai.ChatCompletion.acreate( # Changed to acreate
#         model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.0, 
#         api_key=openai_api_key,
#     )
#     code_blocks = response.choices[0].message.content
#     # print("[i] Extracted code blocks:", code_blocks)
#     return extract_first_code_block(code_blocks) or 'print("no code found")'

# def extract_first_code_block(text):
#     m = re.search(r"```(?:python)?\s*(.*?)```", text, re.DOTALL)
#     return m.group(1).strip() if m else text.strip()

# def write_test_code(repo_dir, code):
#     path = Path(repo_dir) / 'test_code.py'
#     with open(path, 'w') as f:
#         f.write(code)
#     return path

# def parse_requirements_txt(doc):
#     # Extract requirements.txt content if present
#     # Assuming 'doc' is a string containing the content of all ingested files,
#     # and each file's content is prefixed by its name.
#     # A more robust way would be if `ingest` returned structured data.
#     # For now, we search for a section starting with "File: requirements.txt" or similar.
#     # If `ingest` concatenates files, we need a reliable delimiter.
#     # Let's assume `gitingest` provides file content distinctly or `doc` contains identifiable sections.
#     # A simple regex for "requirements.txt" content block:
#     req_content = ""
#     # Try to find requirements.txt content. This might need adjustment based on `ingest` output format.
#     # If `docs` is a dictionary mapping filenames to content:
#     if isinstance(doc, dict) and "requirements.txt" in doc:
#         req_content = doc["requirements.txt"]
#     # If `docs` is a single string with file contents concatenated:
#     else:
#         # This regex assumes a simple structure like "File: requirements.txt\n<content>"
#         # or just the content of requirements.txt if it's the only thing related to it.
#         # This part is speculative without knowing the exact format of `docs` from `ingest`
#         # when multiple patterns are used.
#         # For gitingest, `content` is a string where files are concatenated,
#         # prefixed by "File: <filepath>\n---\n<content>\n---".
#         match = re.search(r"File:.*?requirements\.txt\n---\n(.*?)\n---", doc, re.DOTALL | re.IGNORECASE)
#         if match:
#             req_content = match.group(1)

#     requirements = []
#     if req_content:
#         for line in req_content.split('\n'):
#             line = line.strip()
#             if line and not line.startswith('#'):
#                 # Remove version specifiers, comments, and extras
#                 package = re.split(r'[=<>!~#\[]', line)[0].strip()
#                 if package:
#                     requirements.append(package)
#     return requirements

# async def install_all_dependencies(container, doc, repo_dir="/app", repo_name=""):
#     # First get dependencies from requirements.txt if present in ingested docs
#     parsed_deps = parse_requirements_txt(doc)
    
#     # Then get additional dependencies from content analysis using LLM
#     prompt = (
#         "Based on the content of the file below (primarily README.md, but also consider other context if provided), "
#         "create a list of Python packages to be added in a requirements.txt file. "
#         "List only the package names, one per line. Only include packages explicitly mentioned for installation or clearly imported and used. "
#         "Do not include any dashes or version numbers. The content of the file is: " + doc
#     )
#     # Use async version of OpenAI call
#     response = await openai.ChatCompletion.acreate( # Changed to acreate
#         model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.0,
#         api_key=openai_api_key,
#     )
    
#     llm_deps = [dep.strip() for dep in response.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
    
#     # Combine both sources of dependencies and remove duplicates
#     dependencies = list(dict.fromkeys(parsed_deps + llm_deps))
#     requirements_content = "\n".join(dependencies)
    
#     if not requirements_content.strip():
#         print("[i] No dependencies found to install.")
#         requirements_successful[repo_name] = container.id
#         return True

#     # Run blocking docker exec_run in a thread
#     await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
#     print(f"[i] Created requirements.txt in {container.name} with contents:\n{requirements_content}")
    
#     # Run blocking docker exec_run in a thread
#     result = await asyncio.to_thread(container.exec_run, f"pip install -r requirements.txt", workdir=repo_dir)
#     print(f"[i] Installing dependencies from requirements.txt in {repo_dir}")
#     print(f"[i] Output for {repo_dir}:", result.output.decode()[:200], '\n \nENDING WITH:', result.output.decode()[-400:]) #only print first 200 characters
    
#     # Check if requirements installation was successful
#     if result.exit_code == 0:
#         requirements_successful[repo_name] = container.id
#         return True
#     else:
#         error_message = f"Requirements installation failed: {result.output.decode()}"
#         error_log[repo_name] = error_message
#         print(f"[!] Requirements installation failed for {repo_name}: {error_message}")
#         return False

# async def extract_missing_dependencies(traceback_text):
#     """Extract missing dependencies from traceback using OpenAI"""
#     prompt = (
#         "Given the following Python traceback error, identify any missing Python packages that need to be installed. "
#         "Return only the package names, one per line, without any version numbers or additional text. "
#         "If no missing packages can be identified, return an empty response. "
#         "The traceback is: " + traceback_text
#     )
    
#     response = await openai.ChatCompletion.acreate(
#         model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.0,
#         api_key=openai_api_key,
#     )
    
#     missing_deps = [dep.strip() for dep in response.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
#     return missing_deps

# async def install_missing_dependencies(container, missing_deps, repo_dir="/app"):
#     """Install missing dependencies identified from traceback"""
#     if not missing_deps:
#         return False
    
#     # Read current requirements.txt
#     result = await asyncio.to_thread(container.exec_run, f"cat {repo_dir}/requirements.txt", workdir=repo_dir)
#     current_requirements = result.output.decode().strip().split('\n') if result.exit_code == 0 else []
    
#     # Add missing dependencies that aren't already in requirements
#     new_deps = [dep for dep in missing_deps if dep not in current_requirements]
    
#     if new_deps:
#         all_requirements = current_requirements + new_deps
#         requirements_content = "\n".join(filter(None, all_requirements))  # Filter out empty strings
        
#         # Update requirements.txt
#         await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
#         print(f"[i] Updated requirements.txt with missing dependencies: {new_deps}")
#         print("Full requirements.txt content:", requirements_content)
        
#         # Install the new dependencies
#         for dep in new_deps:
#             result = await asyncio.to_thread(container.exec_run, f"pip install {dep}", workdir=repo_dir)
#             print(f"[i] Installing {dep}: {result.output.decode()}")
#         return True
#     return False

# async def run_test_code(container, repo_dir="/app", repo_name=""):
#     # Run blocking docker exec_run in a thread
#     result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
#     output_str = result.output.decode()
#     print(f"[o] Output for {repo_dir} ({container.name}):\n", output_str)

#     if "Traceback (most recent call last):" in output_str:#output_str.strip().startswith("Traceback"):
#         print(f"[!] Traceback detected in {container.name}. Attempting to fix missing dependencies.")
        
#         # Track that requirements were successful but code initially failed
#         if repo_name in requirements_successful:
#             requirements_successful_code_failed[repo_name] = container.id
        
#         # Try up to 2 times to install missing dependencies
#         dependency_install_successful = False
#         for attempt in range(2):
#             print(f"[i] Dependency installation attempt {attempt + 1}/2")
            
#             # Extract missing dependencies from the traceback
#             missing_deps = await extract_missing_dependencies(output_str)
            
#             if missing_deps:
#                 print(f"[i] Found potential missing dependencies: {missing_deps}")
#                 install_success = await install_missing_dependencies(container, missing_deps, repo_dir)
                
#                 if install_success:
#                     dependency_install_successful = True
#                     # Retry running the test code
#                     print(f"[i] Retrying test code execution in {container.name}")
#                     retry_result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
#                     retry_output = retry_result.output.decode()
#                     print(f"[o] Retry output for ({container.name}):\n", retry_output)
                    
#                     # Check if retry was successful
#                     if not "Traceback" in retry_output:#retry_output.strip().startswith("Traceback"):
#                         print(f"[✓] Test code succeeded after installing missing dependencies in {container.name}")
#                         code_successful[repo_name] = container.id
#                         # Remove from failed dict if it was there
#                         if repo_name in requirements_successful_code_failed:
#                             del requirements_successful_code_failed[repo_name]
#                         return True
#                     elif 'ModuleNotFoundError' not in retry_output:
#                         print(f"[!] Dependencies installed successfully in {container.name} but other error occurred")
#                         error_log[repo_name] = f"Code execution error after dependency installation: {retry_output}"
#                         return False
#                     else:
#                         # Update output_str for next iteration if there's another attempt
#                         output_str = retry_output
#             else:
#                 print(f"[!] No missing dependencies identified from traceback on attempt {attempt + 1}.")
#                 break  # No deps found, don't continue trying
        
#         # If we've exhausted attempts and still have dependency issues, delete container
#         if dependency_install_successful and output_str.strip().startswith("Traceback") and 'ModuleNotFoundError' in output_str:
#             print(f"[!] Unable to resolve dependencies after 2 attempts. Deleting container {container.name}.")
#             error_log[repo_name] = f"Unresolvable dependency issues after 2 attempts: {output_str}"
#             try:
#                 await asyncio.to_thread(container.stop)
#                 await asyncio.to_thread(container.remove)
#                 print(f"[!] Deleted container {container.name} due to unresolvable dependency issues.")
#             except docker.errors.APIError as e:
#                 print(f"[!] Error stopping/removing container {container.name}: {e}")
#             return False
#         elif not dependency_install_successful:
#             error_log[repo_name] = f"No dependencies could be extracted from traceback: {output_str}"
#             print(f"[!] No dependencies could be extracted. Keeping container {container.name} for manual inspection.")
#             return False
#         else:
#             error_log[repo_name] = f"Code execution error (non-dependency): {output_str}"
#             print(f"[!] Non-dependency error in {container.name}. Keeping container for inspection.")
#             return False
            
#     elif output_str.strip().startswith("File not found"):
#         print(f"[!] File not found error in {container.name}. Keeping container for inspection.")
#         error_log[repo_name] = f"File not found error: {output_str}"
#         return False
    
#     # Test code succeeded
#     code_successful[repo_name] = container.id
#     # Remove from failed dict if it was there
#     if repo_name in requirements_successful_code_failed:
#         del requirements_successful_code_failed[repo_name]
#     return True

# async def full_pipeline(git_url):
#     repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    
#     try:
#         repo_dir = await setup_repo(git_url)
#         summary, tree, docs = await ingest_repo(repo_dir) # Added await
#         # print(f"[i] Ingested documents for {git_url}:\n{docs}\n") # For debugging doc format
#         # print("[i] Summary:", summary)
#         # print("[i] Tree:", tree)
#         code = await extract_main_code(docs) # Added await
        
#         # If no code found, return early without creating container
#         if code.strip() == 'print("no code found")':
#             print(f"No usable code found for {git_url}, skipping container creation")
#             error_log[repo_name] = "No usable code found"
#             return None
            
#         write_test_code(repo_dir, code) # Sync, assumed fast
        
#         container_name = f"repo_{repo_name}"
        
#         # Check if container already exists and delete it
#         try:
#             existing_container = await asyncio.to_thread(docker_client.containers.get, container_name)
#             if existing_container:
#                 print(f"Found existing container with name: {container_name}. Deleting it.")
#                 await asyncio.to_thread(existing_container.stop)
#                 await asyncio.to_thread(existing_container.remove)
#                 print(f"Deleted container with name: {container_name}")
#         except docker.errors.NotFound:
#             print(f"No existing container found with name: {container_name}. Proceeding to create a new one.")
#         except docker.errors.APIError as e:
#             print(f"Error checking for existing container {container_name}: {e}")
#             # Decide if you want to stop or continue. For now, let's try to continue.
#             pass

#         # Run blocking docker containers.run in a thread using Linux-based Python image
#         container = await asyncio.to_thread(
#             docker_client.containers.run,
#             "python:3.10-slim",
#             name=container_name,
#             detach=True,
#             tty=True,
#             volumes={str(repo_dir.absolute()): {'bind': '/app', 'mode': 'rw'}},
#             working_dir='/app',
#             platform='linux/amd64' # Specify platform for consistency
#         )
        
#         print(f"[i] Created Docker container: {container_name}")
        
#         container_info = {
#             "repo": repo_name,
#             "container_id": container.id,
#             "container_name": container_name
#         }
        
#         # Save json inside repo directory
#         with open(repo_dir / f"container_{repo_name}.json", "w") as f:
#             json.dump(container_info, f)

#         # Install dependencies only after container is confirmed running
#         requirements_success = await install_all_dependencies(container, docs, "/app", repo_name)
        
#         if not requirements_success:
#             print(f"Requirements installation failed for {git_url}. Keeping container {container_name} for inspection.")
#             return container
        
#         test_successful = await run_test_code(container, "/app", repo_name)
        
#         return container
        
#     except Exception as e:
#         error_log[repo_name] = f"Pipeline error: {str(e)}"
#         print(f"Pipeline error for {git_url}: {e}")
#         return None

# async def run_all_pipelines():
#     # Ensure openai_api_key is set before running, e.g.
#     # openai.api_key = os.getenv("OPENAI_API_KEY")
#     # Or ensure it's globally available if not passed explicitly to functions.
#     # For this example, I've modified extract_main_code and install_all_dependencies
#     # to use os.getenv("OPENAI_API_KEY") directly.

#     # Run pipelines sequentially, one at a time
#     successful_containers = []
#     container_ids = {}

#     Path('repos').mkdir(exist_ok=True)

#     for i, url in enumerate(tqdm(github_links_subset, desc="Processing repositories")):
#         time.sleep(10)
#         print(f"\n{'='*80}")
#         print(f"Processing repository: {url}")
#         print(f"{'='*80}")
        
#         try:
#             result = await full_pipeline(url)
#             repo_name_for_id = url.rstrip('/').split('/')[-1].replace('.git', '')
            
#             if isinstance(result, docker.models.containers.Container):
#                 successful_containers.append(result)
#                 container_ids[repo_name_for_id] = result.id
#                 print(f"Pipeline for {url} completed. Container ID: {result.id}")
#             elif result is None:
#                 # This case is handled if full_pipeline returns None (e.g. test code failed)
#                 print(f"Pipeline for {url} did not complete successfully or container was removed.")
#             else:
#                 print(f"Pipeline for {url} returned unexpected result: {result}")
                
#         except Exception as e:
#             repo_name_for_id = url.rstrip('/').split('/')[-1].replace('.git', '')
#             error_log[repo_name_for_id] = f"Pipeline error: {str(e)}"
#             print(f"Pipeline for {url} failed with an exception: {e}")
#             # Optionally, log the traceback:
#             # import traceback
#             # traceback.print_exception(type(e), e, e.__traceback__)

#         # Save tracking information every 5 URLs
#         if (i + 1) % 5 == 0 or i == len(github_links_subset) - 1:
#             print(f"\nSaving tracking information after processing {i + 1} repositories...")
            
#             with open("repos/all_container_ids.json", "w") as f:
#                 json.dump(container_ids, f)
            
#             with open("repos/error_log.json", "w") as f:
#                 json.dump(error_log, f, indent=2)
            
#             with open("repos/requirements_successful.json", "w") as f:
#                 json.dump(requirements_successful, f, indent=2)
            
#             with open("repos/code_successful.json", "w") as f:
#                 json.dump(code_successful, f, indent=2)
            
#             with open("repos/requirements_successful_code_failed.json", "w") as f:
#                 json.dump(requirements_successful_code_failed, f, indent=2)
    
#     print(f"\nFinal results:")
#     print(f"Successfully processed {len(successful_containers)} repositories.")
#     print(f"Requirements successful: {len(requirements_successful)} repositories")
#     print(f"Code successful: {len(code_successful)} repositories")
#     print(f"Requirements successful but code failed: {len(requirements_successful_code_failed)} repositories")
#     print(f"Errors logged: {len(error_log)} repositories")
#     print(f"Container IDs saved to repos/all_container_ids.json: {container_ids}")
#     print("Error log saved to repos/error_log.json")
#     print("Requirements successful saved to repos/requirements_successful.json")
#     print("Code successful saved to repos/code_successful.json")
#     print("Requirements successful but code failed saved to repos/requirements_successful_code_failed.json")

# # Run in Jupyter/IPython:
# await run_all_pipelines()

In [ ]:
# Run till here 190 repos 

Pip and npm installs

In [ ]:
import asyncio
import json
import openai, subprocess, os, re
from gitingest import ingest
import docker
from pathlib import Path
from tqdm import tqdm
import time

# Initialize Docker client
docker_client = docker.from_env()

# Global tracking dictionaries
error_log = {}  # repo_name -> error_string
requirements_successful = {}  # repo_name -> container_id
code_successful = {}  # repo_name -> container_id
requirements_successful_code_failed = {}  # repo_name -> container_id

# Assumed to be globally available or set via os.getenv("OPENAI_API_KEY")
# openai_api_key = os.getenv("OPENAI_API_KEY") # Example if needed

def append_error(repo_name, error_type_prefix, message):
    """Helper function to append errors to the global error_log."""
    full_message = f"{error_type_prefix}: {message}"
    existing_error = error_log.get(repo_name)
    if existing_error:
        error_log[repo_name] = f"{existing_error}; {full_message}"
    else:
        error_log[repo_name] = full_message
    # Print a truncated version of the message to keep logs readable
    print(f"[!] {error_type_prefix} for {repo_name}: {message[:500]}{'...' if len(message) > 500 else ''}")


async def setup_repo(git_url, base_dir='repos'):
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    print("Repo name is:", repo_name)
    repo_dir = Path(base_dir) / repo_name
    # print("Repo dir is:", repo_dir)
    if not repo_dir.exists():
        # Run blocking subprocess in a thread
        await asyncio.to_thread(subprocess.run, ['git', 'clone', git_url, str(repo_dir)], check=True)
    return repo_dir

async def ingest_repo(repo_dir): # Made async
    # Run blocking ingest call in a thread
    # Added "package.json" to include_patterns
    summary, tree, content = await asyncio.to_thread(
        ingest, str(repo_dir), 
        include_patterns={"README.md", "requirements.txt", "package.json"}
    )
    return summary, tree, content

async def extract_main_code(documents): # Made async
    prompt = (
        "Given the following content from a GitHub repo, extract or synthesize a minimal executable .py script "
        "that demonstrates how to use the package or run its main functionality."
        "The output should be valid Python code for a `test_code.py` file. If there's nothing executable, just output "
        "`print(\"no code found\")`. The content is: "
        f"{documents} \n \n"
    )
    # Use async version of OpenAI call
    response = await openai.ChatCompletion.acreate( # Changed to acreate
        model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0, 
        api_key=openai_api_key, # Make sure openai_api_key is defined in the global scope or passed
    )
    code_blocks = response.choices[0].message.content
    # print("[i] Extracted code blocks:", code_blocks)
    return extract_first_code_block(code_blocks) or 'print("no code found")'

def extract_first_code_block(text):
    m = re.search(r"```(?:python)?\s*(.*?)```", text, re.DOTALL)
    return m.group(1).strip() if m else text.strip()

def write_test_code(repo_dir, code):
    path = Path(repo_dir) / 'test_code.py'
    with open(path, 'w') as f:
        f.write(code)
    return path

def parse_requirements_txt(doc):
    req_content = ""
    match = re.search(r"File:.*?requirements\.txt\n---\n(.*?)\n---", doc, re.DOTALL | re.IGNORECASE)
    if match:
        req_content = match.group(1)

    requirements = []
    if req_content:
        for line in req_content.split('\n'):
            line = line.strip()
            if line and not line.startswith('#'):
                package = re.split(r'[=<>!~#\[]', line)[0].strip()
                if package:
                    requirements.append(package)
    return requirements

async def install_all_dependencies(container, doc, repo_dir="/app", repo_name=""):
    overall_deps_successful = True
    
    # --- NPM Dependency Installation ---
    has_package_json = False
    # Check if package.json content was ingested
    match_pkg_json = re.search(r"File:.*?package\.json\n---\n", doc, re.IGNORECASE | re.DOTALL)
    if match_pkg_json:
        has_package_json = True
        print(f"[i] Found package.json content in ingested docs for {repo_name}.")

    if has_package_json:
        print(f"[i] Attempting npm install for {repo_name}.")
        
        # Install Node.js and npm. python:slim images run as root.
        print(f"[i] Installing Node.js and npm in {container.name}...")
        # Using -qq for quieter output from apt-get
        install_npm_cmd = "apt-get update -qq && apt-get install -y -qq nodejs npm"
        
        npm_setup_result = await asyncio.to_thread(container.exec_run, f"bash -c '{install_npm_cmd}'")
        
        if npm_setup_result.exit_code == 0:
            print(f"[i] Node.js and npm installed successfully in {container.name}.")
            
            # Run npm install
            npm_install_cmd = "npm install"
            print(f"[i] Running '{npm_install_cmd}' in {repo_dir} for {repo_name}...")
            npm_result = await asyncio.to_thread(container.exec_run, npm_install_cmd, workdir=repo_dir)
            
            npm_output = npm_result.output.decode()
            print(f"[i] npm install output for {repo_name} (first 200 chars):\n{npm_output[:200]}")
            if len(npm_output) > 600: # Show tail if output is long
                 print(f"[i] npm install output for {repo_name} (last 400 chars):\n{npm_output[-400:]}")


            if npm_result.exit_code != 0:
                overall_deps_successful = False
                append_error(repo_name, "NPM_INSTALL_ERROR", npm_output)
            else:
                print(f"[✓] npm install successful for {repo_name}.")
        else:
            overall_deps_successful = False
            append_error(repo_name, "NPM_SETUP_ERROR", npm_setup_result.output.decode())
    else:
        print(f"[i] No package.json found or ingested for {repo_name}. Skipping npm install.")

    # --- Python Dependency Installation ---
    if not overall_deps_successful:
        print(f"[!] Skipping Python dependencies due to earlier npm failure for {repo_name}.")
    else:
        parsed_deps = parse_requirements_txt(doc)
        
        prompt_python_deps = (
            "Based on the content of the file below (primarily README.md, but also consider other context if provided), "
            "create a list of Python packages to be added in a requirements.txt file. "
            "List only the package names, one per line. Only include packages explicitly mentioned for installation or clearly imported and used. "
            "Do not include any dashes or version numbers. The content of the file is: " + doc
        )
        response_python_deps = await openai.ChatCompletion.acreate(
            model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18",
            messages=[{"role": "user", "content": prompt_python_deps}],
            temperature=0.0,
            api_key=openai_api_key, # Make sure openai_api_key is defined
        )
        llm_deps = [dep.strip() for dep in response_python_deps.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
        
        dependencies = list(dict.fromkeys(parsed_deps + llm_deps))
        requirements_content = "\n".join(dependencies)
        
        if not requirements_content.strip():
            print(f"[i] No Python dependencies found to install for {repo_name}.")
        else:
            await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
            print(f"[i] Created requirements.txt in {container.name} with contents:\n{requirements_content}")
            
            print(f"[i] Installing Python dependencies from requirements.txt in {repo_dir} for {repo_name}...")
            pip_result = await asyncio.to_thread(container.exec_run, f"pip install -r requirements.txt", workdir=repo_dir)
            
            pip_output = pip_result.output.decode()
            print(f"[i] Pip install output for {repo_dir}:", pip_output[:200], '\n \nENDING WITH:', pip_output[-400:])
            
            if pip_result.exit_code != 0:
                overall_deps_successful = False
                append_error(repo_name, "PIP_INSTALL_ERROR", pip_output)
            else:
                print(f"[✓] Pip install successful for {repo_name}.")

    if overall_deps_successful:
        requirements_successful[repo_name] = container.id
        print(f"[✓] All dependency installations successful for {repo_name}.")
    else:
        if repo_name in requirements_successful: # Should not happen if logic is correct, but as safeguard
            del requirements_successful[repo_name]
        print(f"[!] One or more dependency installation steps failed for {repo_name}.")
        # Specific error details are already in error_log via append_error

    return overall_deps_successful

async def extract_missing_dependencies(traceback_text):
    """Extract missing dependencies from traceback using OpenAI"""
    prompt = (
        "Given the following Python traceback error, identify any missing Python packages that need to be installed. "
        "Return only the package names, one per line, without any version numbers or additional text. "
        "If no missing packages can be identified, return an empty response. "
        "The traceback is: " + traceback_text
    )
    
    response = await openai.ChatCompletion.acreate(
        model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        api_key=openai_api_key, # Make sure openai_api_key is defined
    )
    
    missing_deps = [dep.strip() for dep in response.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
    return missing_deps

async def install_missing_dependencies(container, missing_deps, repo_dir="/app"):
    """Install missing dependencies identified from traceback"""
    if not missing_deps:
        return False
    
    # Read current requirements.txt
    result_cat_req = await asyncio.to_thread(container.exec_run, f"cat {repo_dir}/requirements.txt", workdir=repo_dir)
    current_requirements = result_cat_req.output.decode().strip().split('\n') if result_cat_req.exit_code == 0 else []
    
    # Add missing dependencies that aren't already in requirements
    new_deps = [dep for dep in missing_deps if dep not in current_requirements]
    
    if new_deps:
        all_requirements = current_requirements + new_deps
        requirements_content = "\n".join(filter(None, all_requirements))  # Filter out empty strings
        
        # Update requirements.txt
        await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
        print(f"[i] Updated requirements.txt with missing dependencies: {new_deps}")
        print("Full requirements.txt content:", requirements_content)
        
        # Install the new dependencies
        install_failed_for_any_dep = False
        for dep in new_deps:
            print(f"[i] Installing missing dependency: {dep}")
            result_pip_install_dep = await asyncio.to_thread(container.exec_run, f"pip install {dep}", workdir=repo_dir)
            print(f"[i] Installing {dep}: {result_pip_install_dep.output.decode()}")
            if result_pip_install_dep.exit_code != 0:
                install_failed_for_any_dep = True
                # Log this specific failure if needed, though the overall run_test_code will capture the next traceback
        return not install_failed_for_any_dep # Return True if all new deps installed successfully
    return False

async def run_test_code(container, repo_dir="/app", repo_name=""):
    # Run blocking docker exec_run in a thread
    result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
    output_str = result.output.decode()
    print(f"[o] Output for {repo_dir} ({container.name}):\n", output_str)

    if "Traceback" in output_str:#output_str.strip().startswith("Traceback"):
        print(f"[!] Traceback detected in {container.name}. Attempting to fix missing dependencies.")
        
        if repo_name in requirements_successful: # This implies initial deps (pip/npm) were okay
            requirements_successful_code_failed[repo_name] = container.id
        
        dependency_install_successful_on_retry = False
        for attempt in range(2): # Try up to 2 times to install missing Python dependencies
            print(f"[i] Python dependency installation attempt {attempt + 1}/2 for {repo_name}")
            
            missing_deps = await extract_missing_dependencies(output_str)
            
            if missing_deps:
                print(f"[i] Found potential missing Python dependencies: {missing_deps} for {repo_name}")
                install_success_for_missing = await install_missing_dependencies(container, missing_deps, repo_dir)
                
                if install_success_for_missing:
                    dependency_install_successful_on_retry = True
                    print(f"[i] Retrying test code execution in {container.name}")
                    retry_result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
                    retry_output = retry_result.output.decode()
                    print(f"[o] Retry output for {repo_name} ({container.name}):\n", retry_output)
                    
                    if not "Traceback" in retry_output:#retry_output.strip().startswith("Traceback"):
                        print(f"[✓] Test code succeeded after installing missing Python dependencies in {container.name}")
                        code_successful[repo_name] = container.id
                        if repo_name in requirements_successful_code_failed:
                            del requirements_successful_code_failed[repo_name]
                        return True # Successfully fixed and run
                    elif 'ModuleNotFoundError' not in retry_output: # Different error after installing deps
                        append_error(repo_name, "CODE_EXEC_ERROR_POST_FIX", retry_output)
                        print(f"[!] Dependencies installed, but another error occurred in {container.name}.")
                        return False # Error, but not a ModuleNotFoundError
                    else: # Still ModuleNotFoundError or other Traceback
                        output_str = retry_output # Use new traceback for next attempt
                else: # install_missing_dependencies returned False (failed to install one of the missing_deps)
                    print(f"[!] Failed to install one or more identified missing dependencies on attempt {attempt + 1} for {repo_name}.")
                    # No need to break, LLM might find different deps next time if traceback changes, but unlikely for same error
            else: # No missing deps identified by LLM
                print(f"[!] No missing Python dependencies identified from traceback on attempt {attempt + 1} for {repo_name}.")
                break # Stop trying if LLM finds nothing
        
        # Post-loop evaluation
        final_error_message = f"Traceback after {attempt+1} attempts: {output_str}"
        if dependency_install_successful_on_retry and output_str.strip().startswith("Traceback") and 'ModuleNotFoundError' in output_str:
            # This case means we successfully installed *some* deps, but it still failed with ModuleNotFoundError
            append_error(repo_name, "UNRESOLVABLE_PYTHON_DEPS", final_error_message)
            print(f"[!] Unable to resolve Python dependencies for {repo_name} after multiple attempts. Deleting container {container.name}.")
            try:
                await asyncio.to_thread(container.stop)
                await asyncio.to_thread(container.remove)
                print(f"[!] Deleted container {container.name} due to unresolvable Python dependency issues.")
            except docker.errors.APIError as e:
                print(f"[!] Error stopping/removing container {container.name}: {e}")
            return False
        elif not dependency_install_successful_on_retry and output_str.strip().startswith("Traceback"):
            # This means either no deps were found, or installing them failed.
            append_error(repo_name, "CODE_EXEC_TRACEBACK_NO_FIX", final_error_message)
            print(f"[!] Code execution failed with traceback for {repo_name}, and dependency fixes were not fully successful or not identified.")
            return False
        else: # Other non-ModuleNotFoundError tracebacks, or if loop finished due to non-ModuleNotFoundError
            append_error(repo_name, "CODE_EXEC_ERROR_FINAL", output_str)
            print(f"[!] Non-dependency error or unhandled error state in {container.name}. Keeping container for inspection.")
            return False
            
    elif "no code found" in output_str: # Check if the script itself says "no code found"
        print(f"[!] 'no code found' output by test_code.py in {container.name}. This implies successful execution of a placeholder.")
        # This could be treated as a success or a specific type of failure based on requirements.
        # For now, let's assume it's a "successful" run of a script that found nothing.
        code_successful[repo_name] = container.id 
        if repo_name in requirements_successful_code_failed:
            del requirements_successful_code_failed[repo_name]
        return True

    elif output_str.strip().startswith("File not found"): # This is a custom message, not a Python Traceback
        append_error(repo_name, "FILE_NOT_FOUND_ERROR", output_str)
        print(f"[!] File not found error in {container.name}. Keeping container for inspection.")
        return False
    
    # Test code succeeded without issues
    print(f"[✓] Test code executed successfully in {container.name}.")
    code_successful[repo_name] = container.id
    if repo_name in requirements_successful_code_failed:
        del requirements_successful_code_failed[repo_name]
    return True

async def full_pipeline(git_url):
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    
    try:
        repo_dir_host = await setup_repo(git_url) # Path on host
        summary, tree, docs = await ingest_repo(repo_dir_host)
        code = await extract_main_code(docs)
        
        if code.strip() == 'print("no code found")':
            print(f"No usable Python code found by LLM for {git_url}, skipping container creation and execution.")
            append_error(repo_name, "NO_USABLE_CODE_FOUND", "LLM returned 'print(\"no code found\")'")
            # Optionally, still create container if npm dependencies might be relevant for other tests later
            # For now, we stop if no Python code.
            return None 
            
        write_test_code(repo_dir_host, code)
        
        container_name = f"repo_{repo_name.replace('.', '_')}" # Ensure container name is valid
        
        try:
            existing_container = await asyncio.to_thread(docker_client.containers.get, container_name)
            print(f"Found existing container: {container_name}. Stopping and removing it.")
            await asyncio.to_thread(existing_container.stop)
            await asyncio.to_thread(existing_container.remove, v=True) # v=True removes volumes associated with the container
            print(f"Deleted existing container: {container_name}")
        except docker.errors.NotFound:
            print(f"No existing container found with name: {container_name}. Proceeding to create a new one.")
        except docker.errors.APIError as e:
            print(f"Error managing existing container {container_name}: {e}. Attempting to proceed.")
            append_error(repo_name, "DOCKER_CLEANUP_ERROR", str(e))

        repo_dir_container = "/app" # Standardized path inside container
        container = await asyncio.to_thread(
            docker_client.containers.run,
            "python:3.10-slim",
            name=container_name,
            detach=True,
            tty=True, # Keep STDIN open even if not attached, needed for some processes
            volumes={str(repo_dir_host.absolute()): {'bind': repo_dir_container, 'mode': 'rw'}},
            working_dir=repo_dir_container,
            platform='linux/amd64'
        )
        print(f"[i] Created Docker container: {container.id} ({container_name}) for {repo_name}")
        
        container_info = {"repo": repo_name, "container_id": container.id, "container_name": container_name}
        with open(repo_dir_host / f"container_{repo_name}.json", "w") as f:
            json.dump(container_info, f)

        dependencies_installed = await install_all_dependencies(container, docs, repo_dir_container, repo_name)
        
        if not dependencies_installed:
            print(f"[!] Dependency installation failed for {git_url}. Container {container_name} kept for inspection.")
            # Error already logged by install_all_dependencies
            return container # Return container for potential manual inspection
        
        test_execution_successful = await run_test_code(container, repo_dir_container, repo_name)
        
        if not test_execution_successful:
            print(f"[!] Test code execution failed for {git_url}. Container {container_name} kept for inspection.")
            # Error logged by run_test_code
            return container # Return container for potential manual inspection
        
        print(f"[✓] Pipeline successful for {git_url}. Container ID: {container.id}")
        # Optionally stop/remove successful containers if not needed for inspection
        # await asyncio.to_thread(container.stop)
        # await asyncio.to_thread(container.remove, v=True)
        # print(f"[i] Stopped and removed successful container {container_name}")
        return container # Or return None if removing successful ones
        
    except subprocess.CalledProcessError as e:
        append_error(repo_name, "GIT_CLONE_ERROR", str(e))
        print(f"Git clone error for {git_url}: {e}")
        return None
    except Exception as e:
        import traceback
        tb_str = traceback.format_exc()
        append_error(repo_name, "PIPELINE_ERROR", f"{str(e)}\n{tb_str}")
        print(f"Pipeline error for {git_url}: {e}\n{tb_str}")
        # Clean up container if it was created and an error occurred mid-pipeline
        try:
            if 'container' in locals() and container:
                print(f"Attempting to stop and remove container {container.name} due to pipeline error.")
                await asyncio.to_thread(container.stop)
                await asyncio.to_thread(container.remove, v=True)
        except Exception as cleanup_e:
            print(f"Error during cleanup for {container.name}: {cleanup_e}")
        return None

async def run_all_pipelines():
    global openai_api_key # Ensure it's accessible
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        print("Error: OPENAI_API_KEY environment variable not set.")
        return
    openai.api_key = openai_api_key


    successful_containers_map = {} # repo_name -> container_id for truly successful ones
    all_created_container_ids = {} # repo_name -> container_id for all containers created (even if failed later)

    Path('repos').mkdir(exist_ok=True)

    for i, url in enumerate(tqdm(github_links_subset, desc="Processing repositories")):
        # time.sleep(10) # Kept from original, adjust as needed
        print(f"\n{'='*80}")
        print(f"Processing repository {i+1}/{len(github_links_subset)}: {url}")
        print(f"{'='*80}")
        
        repo_name_for_tracking = url.rstrip('/').split('/')[-1].replace('.git', '')
        
        try:
            container_instance = await full_pipeline(url)
            
            if isinstance(container_instance, docker.models.containers.Container):
                all_created_container_ids[repo_name_for_tracking] = container_instance.id
                # Check if it was a "full" success (code ran)
                if repo_name_for_tracking in code_successful:
                    successful_containers_map[repo_name_for_tracking] = container_instance.id
                    print(f"Pipeline for {url} completed successfully. Container ID: {container_instance.id}")
                else:
                    print(f"Pipeline for {url} created container {container_instance.id}, but did not complete all steps successfully.")
            elif container_instance is None:
                # This case means pipeline failed before or during container creation, or container was removed.
                print(f"Pipeline for {url} did not result in a running container or was intentionally stopped.")
            else: # Should not happen
                print(f"Pipeline for {url} returned unexpected result: {container_instance}")
                append_error(repo_name_for_tracking, "UNEXPECTED_PIPELINE_RESULT", str(container_instance))
                
        except Exception as e: # Catch-all for unexpected errors in run_all_pipelines loop itself
            import traceback
            tb_str = traceback.format_exc()
            append_error(repo_name_for_tracking, "RUN_ALL_PIPELINES_LOOP_ERROR", f"{str(e)}\n{tb_str}")
            print(f"Outer loop exception for {url}: {e}")

        # Save tracking information periodically
        if (i + 1) % 1 == 0 or i == len(github_links_subset) - 1: # Save every repo for better recovery
            print(f"\nSaving tracking information after processing {i + 1} repositories...")
            
            with open("repos/all_created_container_ids.json", "w") as f:
                json.dump(all_created_container_ids, f, indent=2)
            with open("repos/error_log.json", "w") as f:
                json.dump(error_log, f, indent=2)
            with open("repos/requirements_successful.json", "w") as f:
                json.dump(requirements_successful, f, indent=2)
            with open("repos/code_successful.json", "w") as f:
                json.dump(code_successful, f, indent=2)
            with open("repos/requirements_successful_code_failed.json", "w") as f:
                json.dump(requirements_successful_code_failed, f, indent=2)
            print("Tracking information saved.")
    
    print(f"\n{'='*20} Final Summary {'='*20}")
    print(f"Total repositories processed: {len(github_links_subset)}")
    print(f"Containers created: {len(all_created_container_ids)}")
    print(f"Dependencies (pip/npm) successfully installed for: {len(requirements_successful)} repositories")
    print(f"Test code executed successfully for: {len(code_successful)} repositories")
    print(f"Dependencies installed but test code failed for: {len(requirements_successful_code_failed)} repositories")
    print(f"Errors logged for: {len(error_log)} repositories (see repos/error_log.json)")
    print(f"All created container IDs saved to repos/all_created_container_ids.json")

# Run in Jupyter/IPython:
# Ensure OPENAI_API_KEY is set in your environment or define openai_api_key globally.
# Example:
# import os
# openai_api_key = os.getenv("OPENAI_API_KEY")
# if not openai_api_key:
#     raise ValueError("OPENAI_API_KEY not set")
# openai.api_key = openai_api_key

await run_all_pipelines()

In [ ]:
Until here the installation is complete. 

### Run code inside the generated Docker container

In [ ]:
# cd Repo2Run

In [ ]:
# !python build_agent/main.py metabase/metabase 2d5d40a9abee16b9d4926e627d300ae764a43dbe ./repo2run/build_agent

In [ ]:
# docker cp my_container:/app/output.txt ~/Desktop/output.txt #copy file from container to desktop
# docker exec -it <container_name> bash #get into container

Parallel run - Faster (first clones repos and then runs into rate limit, even for gpt4o-mini with 200k tokens/min)

In [ ]:
import asyncio
import json
import openai, subprocess, os, re
from gitingest import ingest
import docker
from pathlib import Path

# Initialize Docker client
docker_client = docker.from_env()

# Global tracking dictionaries
error_log = {}  # repo_name -> error_string
requirements_successful = {}  # repo_name -> container_id
code_successful = {}  # repo_name -> container_id
requirements_successful_code_failed = {}  # repo_name -> container_id

async def setup_repo(git_url, base_dir='repos'):
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    print("Repo name is:", repo_name)
    repo_dir = Path(base_dir) / repo_name
    # print("Repo dir is:", repo_dir)
    if not repo_dir.exists():
        # Run blocking subprocess in a thread
        await asyncio.to_thread(subprocess.run, ['git', 'clone', git_url, str(repo_dir)], check=True)
    return repo_dir

async def ingest_repo(repo_dir): # Made async
    # Run blocking ingest call in a thread
    # Changed include_patterns to be a set of strings as expected by gitingest
    summary, tree, content = await asyncio.to_thread(ingest, str(repo_dir), include_patterns={"README.md", "requirements.txt"})
    return summary, tree, content

async def extract_main_code(documents): # Made async
    prompt = (
        "Given the following content from a GitHub repo, extract or synthesize a minimal executable .py script "
        "that demonstrates how to use the package or run its main functionality."
        "The output should be valid Python code for a `test_code.py` file. If there's nothing executable, just output "
        "`print(\"no code found\")`. The content is: "
        f"{documents} \n \n"
    )
    # Use async version of OpenAI call
    response = await openai.ChatCompletion.acreate( # Changed to acreate
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0, 
        api_key=openai_api_key,
    )
    code_blocks = response.choices[0].message.content
    # print("[i] Extracted code blocks:", code_blocks)
    return extract_first_code_block(code_blocks) or 'print("no code found")'

def extract_first_code_block(text):
    m = re.search(r"```(?:python)?\s*(.*?)```", text, re.DOTALL)
    return m.group(1).strip() if m else text.strip()

def write_test_code(repo_dir, code):
    path = Path(repo_dir) / 'test_code.py'
    with open(path, 'w') as f:
        f.write(code)
    return path

def parse_requirements_txt(doc):
    # Extract requirements.txt content if present
    # Assuming 'doc' is a string containing the content of all ingested files,
    # and each file's content is prefixed by its name.
    # A more robust way would be if `ingest` returned structured data.
    # For now, we search for a section starting with "File: requirements.txt" or similar.
    # If `ingest` concatenates files, we need a reliable delimiter.
    # Let's assume `gitingest` provides file content distinctly or `doc` contains identifiable sections.
    # A simple regex for "requirements.txt" content block:
    req_content = ""
    # Try to find requirements.txt content. This might need adjustment based on `ingest` output format.
    # If `docs` is a dictionary mapping filenames to content:
    if isinstance(doc, dict) and "requirements.txt" in doc:
        req_content = doc["requirements.txt"]
    # If `docs` is a single string with file contents concatenated:
    else:
        # This regex assumes a simple structure like "File: requirements.txt\n<content>"
        # or just the content of requirements.txt if it's the only thing related to it.
        # This part is speculative without knowing the exact format of `docs` from `ingest`
        # when multiple patterns are used.
        # For gitingest, `content` is a string where files are concatenated,
        # prefixed by "File: <filepath>\n---\n<content>\n---".
        match = re.search(r"File:.*?requirements\.txt\n---\n(.*?)\n---", doc, re.DOTALL | re.IGNORECASE)
        if match:
            req_content = match.group(1)

    requirements = []
    if req_content:
        for line in req_content.split('\n'):
            line = line.strip()
            if line and not line.startswith('#'):
                # Remove version specifiers, comments, and extras
                package = re.split(r'[=<>!~#\[]', line)[0].strip()
                if package:
                    requirements.append(package)
    return requirements


async def install_all_dependencies(container, doc, repo_dir="/app", repo_name=""):
    # First get dependencies from requirements.txt if present in ingested docs
    parsed_deps = parse_requirements_txt(doc)
    
    # Then get additional dependencies from content analysis using LLM
    prompt = (
        "Based on the content of the file below (primarily README.md, but also consider other context if provided), "
        "create a list of Python packages to be added in a requirements.txt file. "
        "List only the package names, one per line. Only include packages explicitly mentioned for installation or clearly imported and used. "
        "Do not include any dashes or version numbers. The content of the file is: " + doc
    )
    # Use async version of OpenAI call
    response = await openai.ChatCompletion.acreate( # Changed to acreate
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        api_key=openai_api_key,
    )
    
    llm_deps = [dep.strip() for dep in response.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
    
    # Combine both sources of dependencies and remove duplicates
    dependencies = list(dict.fromkeys(parsed_deps + llm_deps))
    requirements_content = "\n".join(dependencies)
    
    if not requirements_content.strip():
        print("[i] No dependencies found to install.")
        requirements_successful[repo_name] = container.id
        return True

    # Run blocking docker exec_run in a thread
    await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
    print(f"[i] Created requirements.txt in {container.name} with contents:\n{requirements_content}")
    
    # Run blocking docker exec_run in a thread
    result = await asyncio.to_thread(container.exec_run, f"pip install -r requirements.txt", workdir=repo_dir)
    print(f"[i] Installing dependencies from requirements.txt in {repo_dir}")
    print(f"[i] Output for {repo_dir}:", result.output.decode()[:200], '\n \nENDING WITH:', result.output.decode()[-400:]) #only print first 200 characters
    
    # Check if requirements installation was successful
    if result.exit_code == 0:
        requirements_successful[repo_name] = container.id
        return True
    else:
        error_message = f"Requirements installation failed: {result.output.decode()}"
        error_log[repo_name] = error_message
        print(f"[!] Requirements installation failed for {repo_name}: {error_message}")
        return False

async def extract_missing_dependencies(traceback_text):
    """Extract missing dependencies from traceback using OpenAI"""
    prompt = (
        "Given the following Python traceback error, identify any missing Python packages that need to be installed. "
        "Return only the package names, one per line, without any version numbers or additional text. "
        "If no missing packages can be identified, return an empty response. "
        "The traceback is: " + traceback_text
    )
    
    response = await openai.ChatCompletion.acreate(
        model="gpt-4.1-2025-04-14", #"gpt-4o-mini-2024-07-18", #
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        api_key=openai_api_key,
    )
    
    missing_deps = [dep.strip() for dep in response.choices[0].message.content.strip().split("\n") if dep.strip() and not ' ' in dep.strip()]
    return missing_deps

async def install_missing_dependencies(container, missing_deps, repo_dir="/app"):
    """Install missing dependencies identified from traceback"""
    if not missing_deps:
        return False
    
    # Read current requirements.txt
    result = await asyncio.to_thread(container.exec_run, f"cat {repo_dir}/requirements.txt", workdir=repo_dir)
    current_requirements = result.output.decode().strip().split('\n') if result.exit_code == 0 else []
    
    # Add missing dependencies that aren't already in requirements
    new_deps = [dep for dep in missing_deps if dep not in current_requirements]
    
    if new_deps:
        all_requirements = current_requirements + new_deps
        requirements_content = "\n".join(filter(None, all_requirements))  # Filter out empty strings
        
        # Update requirements.txt
        await asyncio.to_thread(container.exec_run, f"bash -c 'echo \"{requirements_content}\" > {repo_dir}/requirements.txt'")
        print(f"[i] Updated requirements.txt with missing dependencies: {new_deps}")
        print("Full requirements.txt content:", requirements_content)
        
        # Install the new dependencies
        for dep in new_deps:
            result = await asyncio.to_thread(container.exec_run, f"pip install {dep}", workdir=repo_dir)
            print(f"[i] Installing {dep}: {result.output.decode()}")
        return True
    return False

async def run_test_code(container, repo_dir="/app", repo_name=""):
    # Run blocking docker exec_run in a thread
    result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
    output_str = result.output.decode()
    print(f"[o] Output for {repo_dir} ({container.name}):\n", output_str)

    if output_str.strip().startswith("Traceback"):
        print(f"[!] Traceback detected in {container.name}. Attempting to fix missing dependencies.")
        
        # Track that requirements were successful but code initially failed
        if repo_name in requirements_successful:
            requirements_successful_code_failed[repo_name] = container.id
        
        # Try up to 2 times to install missing dependencies
        dependency_install_successful = False
        for attempt in range(2):
            print(f"[i] Dependency installation attempt {attempt + 1}/2")
            
            # Extract missing dependencies from the traceback
            missing_deps = await extract_missing_dependencies(output_str)
            
            if missing_deps:
                print(f"[i] Found potential missing dependencies: {missing_deps}")
                install_success = await install_missing_dependencies(container, missing_deps, repo_dir)
                
                if install_success:
                    dependency_install_successful = True
                    # Retry running the test code
                    print(f"[i] Retrying test code execution in {container.name}")
                    retry_result = await asyncio.to_thread(container.exec_run, "python test_code.py", workdir=repo_dir)
                    retry_output = retry_result.output.decode()
                    print(f"[o] Retry output for ({container.name}):\n", retry_output)
                    
                    # Check if retry was successful
                    if not retry_output.strip().startswith("Traceback"):
                        print(f"[✓] Test code succeeded after installing missing dependencies in {container.name}")
                        code_successful[repo_name] = container.id
                        # Remove from failed dict if it was there
                        if repo_name in requirements_successful_code_failed:
                            del requirements_successful_code_failed[repo_name]
                        return True
                    elif 'ModuleNotFoundError' not in retry_output:
                        print(f"[!] Dependencies installed successfully in {container.name} but other error occurred")
                        error_log[repo_name] = f"Code execution error after dependency installation: {retry_output}"
                        return False
                    else:
                        # Update output_str for next iteration if there's another attempt
                        output_str = retry_output
            else:
                print(f"[!] No missing dependencies identified from traceback on attempt {attempt + 1}.")
                break  # No deps found, don't continue trying
        
        # If we've exhausted attempts and still have dependency issues, delete container
        if dependency_install_successful and output_str.strip().startswith("Traceback") and 'ModuleNotFoundError' in output_str:
            print(f"[!] Unable to resolve dependencies after 2 attempts. Deleting container {container.name}.")
            error_log[repo_name] = f"Unresolvable dependency issues after 2 attempts: {output_str}"
            try:
                await asyncio.to_thread(container.stop)
                await asyncio.to_thread(container.remove)
                print(f"[!] Deleted container {container.name} due to unresolvable dependency issues.")
            except docker.errors.APIError as e:
                print(f"[!] Error stopping/removing container {container.name}: {e}")
            return False
        elif not dependency_install_successful:
            error_log[repo_name] = f"No dependencies could be extracted from traceback: {output_str}"
            print(f"[!] No dependencies could be extracted. Keeping container {container.name} for manual inspection.")
            return False
        else:
            error_log[repo_name] = f"Code execution error (non-dependency): {output_str}"
            print(f"[!] Non-dependency error in {container.name}. Keeping container for inspection.")
            return False
            
    elif output_str.strip().startswith("File not found"):
        print(f"[!] File not found error in {container.name}. Keeping container for inspection.")
        error_log[repo_name] = f"File not found error: {output_str}"
        return False
    
    # Test code succeeded
    code_successful[repo_name] = container.id
    # Remove from failed dict if it was there
    if repo_name in requirements_successful_code_failed:
        del requirements_successful_code_failed[repo_name]
    return True

async def full_pipeline(git_url):
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    
    try:
        repo_dir = await setup_repo(git_url)
        summary, tree, docs = await ingest_repo(repo_dir) # Added await
        # print(f"[i] Ingested documents for {git_url}:\n{docs}\n") # For debugging doc format
        # print("[i] Summary:", summary)
        # print("[i] Tree:", tree)
        code = await extract_main_code(docs) # Added await
        
        # If no code found, return early without creating container
        if code.strip() == 'print("no code found")':
            print(f"No usable code found for {git_url}, skipping container creation")
            error_log[repo_name] = "No usable code found"
            return None
            
        write_test_code(repo_dir, code) # Sync, assumed fast
        
        container_name = f"repo_{repo_name}"
        
        # Check if container already exists and delete it
        try:
            existing_container = await asyncio.to_thread(docker_client.containers.get, container_name)
            if existing_container:
                print(f"Found existing container with name: {container_name}. Deleting it.")
                await asyncio.to_thread(existing_container.stop)
                await asyncio.to_thread(existing_container.remove)
                print(f"Deleted container with name: {container_name}")
        except docker.errors.NotFound:
            print(f"No existing container found with name: {container_name}. Proceeding to create a new one.")
        except docker.errors.APIError as e:
            print(f"Error checking for existing container {container_name}: {e}")
            # Decide if you want to stop or continue. For now, let's try to continue.
            pass

        # Run blocking docker containers.run in a thread using Linux-based Python image
        container = await asyncio.to_thread(
            docker_client.containers.run,
            "python:3.10-slim",
            name=container_name,
            detach=True,
            tty=True,
            volumes={str(repo_dir.absolute()): {'bind': '/app', 'mode': 'rw'}},
            working_dir='/app',
            platform='linux/amd64' # Specify platform for consistency
        )
        
        print(f"[i] Created Docker container: {container_name}")
        
        container_info = {
            "repo": repo_name,
            "container_id": container.id,
            "container_name": container_name
        }
        
        # Save json inside repo directory
        with open(repo_dir / f"container_{repo_name}.json", "w") as f:
            json.dump(container_info, f)

        # Install dependencies only after container is confirmed running
        requirements_success = await install_all_dependencies(container, docs, "/app", repo_name)
        
        if not requirements_success:
            print(f"Requirements installation failed for {git_url}. Keeping container {container_name} for inspection.")
            return container
        
        test_successful = await run_test_code(container, "/app", repo_name)
        
        return container
        
    except Exception as e:
        error_log[repo_name] = f"Pipeline error: {str(e)}"
        print(f"Pipeline error for {git_url}: {e}")
        return None

async def run_all_pipelines():
    # Ensure openai_api_key is set before running, e.g.
    # openai.api_key = os.getenv("OPENAI_API_KEY")
    # Or ensure it's globally available if not passed explicitly to functions.
    # For this example, I've modified extract_main_code and install_all_dependencies
    # to use os.getenv("OPENAI_API_KEY") directly.

    # Ensure repos directory exists early
    Path('repos').mkdir(exist_ok=True)

    tasks = [full_pipeline(url) for url in github_links_subset]
    # All tasks are launched and awaited together, maintaining parallel execution.
    containers_results = await asyncio.gather(*tasks, return_exceptions=True)
    
    successful_containers = []
    # container_ids will be populated as we process results.
    # It should be a local variable, not assumed global for population here.
    container_ids = {} 

    # Helper function to save all tracking information
    def _save_all_logs():
        # Save all tracking information
        with open("repos/all_container_ids.json", "w") as f:
            json.dump(container_ids, f, indent=2)
        
        # These global dictionaries are assumed to be populated by full_pipeline
        with open("repos/error_log.json", "w") as f:
            json.dump(error_log, f, indent=2)
        
        with open("repos/requirements_successful.json", "w") as f:
            json.dump(requirements_successful, f, indent=2)
        
        with open("repos/code_successful.json", "w") as f:
            json.dump(code_successful, f, indent=2)
        
        with open("repos/requirements_successful_code_failed.json", "w") as f:
            json.dump(requirements_successful_code_failed, f, indent=2)
        
        print("Successfully saved tracking information to 'repos/' directory.")

    for i, result in enumerate(containers_results):
        url = github_links_subset[i]
        # Assuming repo_name is derived consistently for logging keys
        repo_name_for_id = url.rstrip('/').split('/')[-1].replace('.git', '')

        if isinstance(result, docker.models.containers.Container):
            successful_containers.append(result)
            container_ids[repo_name_for_id] = result.id
            print(f"Pipeline for {url} completed. Container ID: {result.id}")
        elif result is None:
            # This case is handled if full_pipeline returns None (e.g. test code failed or other handled error)
            # error_log and other relevant logs should have been populated by full_pipeline.
            print(f"Pipeline for {url} did not complete successfully or container was removed.")
        elif isinstance(result, Exception):
            # This means asyncio.gather caught an unhandled exception from full_pipeline.
            # full_pipeline is expected to log its own errors to error_log.
            # This print is for visibility of exceptions that might not have been caught by full_pipeline's own try-except.
            print(f"Pipeline for {url} failed with an exception: {result}")
            # Optionally, log the traceback:
            # import traceback
            # traceback.print_exception(type(result), result, result.__traceback__)
            # If full_pipeline doesn't guarantee logging for all its exceptions,
            # you might want to add to error_log here:
            # if repo_name_for_id not in error_log: # Avoid overwriting more specific error from full_pipeline
            #     error_log[repo_name_for_id] = f"Unhandled pipeline error: {str(result)}"

        # Save logs every 5 iterations (except for the very last one, which is covered by final save)
        if (i + 1) % 5 == 0 and (i + 1) < len(containers_results):
            print(f"Processed {i + 1}/{len(containers_results)} repositories. Saving intermediate logs...")
            _save_all_logs()
            # Print intermediate summary counts
            print(f"  So far: {len(successful_containers)} successful containers.")
            print(f"  So far: {len(requirements_successful)} requirements successful.")
            print(f"  So far: {len(code_successful)} code successful.")
            print(f"  So far: {len(requirements_successful_code_failed)} reqs successful, code failed.")
            print(f"  So far: {len(error_log)} errors logged.")


    # Final save of all logs after processing all results
    print(f"Finished processing all {len(containers_results)} repositories. Saving final logs...")
    _save_all_logs()
    
    # Final summary prints
    print(f"Successfully processed {len(successful_containers)} repositories.")
    print(f"Requirements successful: {len(requirements_successful)} repositories")
    print(f"Code successful: {len(code_successful)} repositories")
    print(f"Requirements successful but code failed: {len(requirements_successful_code_failed)} repositories")
    print(f"Errors logged: {len(error_log)} repositories")
    
    print(f"Container IDs saved to repos/all_container_ids.json: {container_ids}")
    print("Error log saved to repos/error_log.json")
    print("Requirements successful saved to repos/requirements_successful.json")
    print("Code successful saved to repos/code_successful.json")
    print("Requirements successful but code failed saved to repos/requirements_successful_code_failed.json")

# Run in Jupyter/IPython:
await run_all_pipelines()

In [ ]:
Run the above to get into the container

Run test code inside a container

In [ ]:
import docker
import json
import traceback

# Create the Python script content
fireducks_code = '''import fireducks.pandas as pd

def main():
    # Create a simple DataFrame
    df = pd.DataFrame({
        "A": [1, 30, 3],
        "B": [4, 5, 6]
    })
    print("FireDucks DataFrame:")
    print(df)
    # Perform a simple operation
    print("\\nSum of column A:", df["A"].sum())

if __name__ == "__main__":
    main()
'''

# Minimal debugging for containers and run the test files
git_urls = [
    "https://github.com/fireducks-dev/fireducks/",
    "https://github.com/astral-sh/uv"
]

client = docker.from_env()

for git_url in git_urls:
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    container_info_file = f"container_{repo_name}.json"
    
    print(f"\n=== Debugging and Running {repo_name} ===")
    
    try:
        with open(container_info_file) as f:
            container_data = json.load(f)
        
        container_id = container_data.get('container_id')
        print(f"Container ID: {container_id}")
        
        if container_id:
            try:
                container = client.containers.get(container_id)
                print(f"Container status: {container.status}")
                print(f"Container name: {container.name}")
                
                # Simple test command
                result = container.exec_run("echo 'Container is responsive'")
                print(f"Test command output: {result.output.decode().strip()}")
                print(f"Test command exit code: {result.exit_code}")
                
                # Run the test files in the workdir app
                if repo_name == "fireducks":
                    print(f"\n--- Running test_fireducks for {repo_name} ---")
                    result = container.exec_run("python test_fireducks.py")
                    print(f"test_fireducks output:\n{result.output.decode().strip()}")
                    print(f"test_fireducks exit code: {result.exit_code}")
                    
                    # Also run the fireducks_code variable
                    print(f"\n--- Running fireducks_code variable for {repo_name} ---")
                    try:
                        # Use single quotes for the -c argument to avoid issues with double quotes in fireducks_code
                        command = f"python -c '{fireducks_code}'"
                        result = container.exec_run(command) # Re-assign result, matches existing pattern
                        
                        output_str = result.output.decode('utf-8', errors='replace').strip()
                        exit_code = result.exit_code

                        print(f"fireducks_code output:\n{output_str}")
                        print(f"fireducks_code exit code: {exit_code}")
                        
                    except docker.errors.APIError as api_e:
                        print(f"Docker API error during fireducks_code execution for {repo_name}: {api_e}")
                        traceback.print_exc()
                    except Exception as exec_e:
                        print(f"Error during fireducks_code execution for {repo_name}: {exec_e}")
                        traceback.print_exc()
                        
                elif repo_name == "uv":
                    print(f"\n--- Running test for {repo_name} ---")
                    result = container.exec_run("python test_uv.py")
                    print(f"test output:\n{result.output.decode().strip()}")
                    print(f"test exit code: {result.exit_code}")
                
            except docker.errors.NotFound:
                print(f"Container {container_id} not found")
            except Exception as e:
                print(f"Error accessing container: {e}")
                traceback.print_exc()
        else:
            print("No container_id found")
            
    except FileNotFoundError:
        print(f"Container info file {container_info_file} not found")
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

### Sandboxes - Not persistent, have to rerun once restart session

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import json
import openai, subprocess, os, re
from gitingest import ingest
from e2b_code_interpreter import Sandbox
from pathlib import Path

async def setup_repo(git_url, base_dir='sandboxes'):
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    print("Repo name is:", repo_name)
    sandbox_dir = Path(base_dir) / repo_name
    print("Sandbox dir is:", sandbox_dir)
    if not sandbox_dir.exists():
        subprocess.run(['git', 'clone', git_url, str(sandbox_dir)], check=True)
    return sandbox_dir

def ingest_repo(sandbox_dir):
    summary, tree, content = ingest(str(sandbox_dir), include_patterns="README.md")
    return summary, tree, content

def extract_main_code(documents):
    prompt = (
        "Given the following content from a GitHub repo, extract or synthesize a minimal executable .py script "
        "that demonstrates how to use the package or run its main functionality."
        "The output should be valid Python code for a `main.py` file. If there's nothing executable, just output "
        "`print(\"no code found\")`. The content is: "
        f"{documents} \n \n"
    )
    response = openai.ChatCompletion.create(
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0, 
        api_key=openai_api_key,
    )
    code_blocks = response.choices[0].message.content
    print("[i] Extracted code blocks:", code_blocks)
    return extract_first_code_block(code_blocks) or 'print("no code found")'

def extract_first_code_block(text):
    m = re.search(r"```(?:python)?\s*(.*?)```", text, re.DOTALL)
    return m.group(1).strip() if m else text.strip()

def write_main_py(sandbox_dir, code):
    path = Path(sandbox_dir) / 'main.py'
    with open(path, 'w') as f:
        f.write(code)
    return path

async def install_all_dependencies(sandbox, doc, repo_dir="/"):
    prompt = (
        "Based on the content of the file below, create a list of python packages to be added in a requirements.txt file."
        "List only the package names, one per line. Only include mentioned to be installed in the content of the file. The content of the file is: " + doc
    )
    response = openai.ChatCompletion.create(
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        api_key=openai_api_key,
    )
    
    # Create requirements.txt from OpenAI response
    dependencies = response.choices[0].message.content.strip().split("\n")
    requirements_content = "\n".join(dependencies)
    
    # Write requirements.txt to sandbox
    sandbox.files.write("/requirements.txt", requirements_content)
    print("[i] Created requirements.txt with contents:\n", requirements_content)
    
    # Install dependencies
    result = sandbox.commands.run("pip install -r requirements.txt", cwd=repo_dir)
    print("[i] Installing dependencies from requirements.txt")
    print("[i] STDOUT:", result.stdout)
    print("[i] STDERR:", result.stderr or "(None)")

    # if "pyproject.toml" in file_names:
    #     result = sandbox.commands.run("pip install .", cwd=repo_dir)
    #     print("[i] Installing package from pyproject.toml")
    #     print("[i] STDOUT:", result.stdout)
    #     print("[i] STDERR:", result.stderr or "(None)")

async def run_main(sandbox, repo_dir):
    result = sandbox.commands.run("python main.py", cwd=repo_dir)
    print("[o] Output:\n", result.stdout)
    print("[o] Errors:\n", result.stderr)

async def full_pipeline(git_url):
    repo_dir = await setup_repo(git_url)
    summary, tree, docs = ingest_repo(repo_dir)
    print("[i] Ingested documents:", len(docs), docs, "\n \n")
    print("[i] Summary:", summary)
    print("[i] Tree:", tree)
    code = extract_main_code(docs)
    write_main_py(repo_dir, code)
    
    # Create sandbox
    sb = Sandbox(template="base")
    
    # Save sandbox ID to file
    sandbox_id = sb.sandbox_id
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    sandbox_info = {
        "repo": repo_name,
        "sandbox_id": sandbox_id
    }
    
    with open(f"sandbox_{repo_name}.json", "w") as f:
        json.dump(sandbox_info, f)

    # Upload files to sandbox
    for file in Path(repo_dir).rglob("*"):
        if file.is_file():
            rel_path = file.relative_to(repo_dir).as_posix()
            with open(file, "rb") as f:
                content = f.read()
            sb.files.write(f"/{rel_path}", content)

    await install_all_dependencies(sb, docs, "/")
    await run_main(sb, "/")
 # To run this in a Jupyter or IPython environment:
# await full_pipeline("https://github.com/fireducks-dev/fireducks/")   
    return sb

# Git URLs to process
git_urls = [
    "https://github.com/fireducks-dev/fireducks/",
    "https://github.com/astral-sh/uv"
    # Add more git URLs here
]

async def run_all_pipelines():
    tasks = [full_pipeline(url) for url in git_urls]
    sandboxes = await asyncio.gather(*tasks)
    
    # Save all sandbox IDs to a single file
    sandbox_ids = {
        url.rstrip('/').split('/')[-1].replace('.git', ''): sb.sandbox_id 
        for url, sb in zip(git_urls, sandboxes)
    }
    
    with open("all_sandbox_ids.json", "w") as f:
        json.dump(sandbox_ids, f)

# Run in Jupyter/IPython:
await run_all_pipelines()

In [ ]:
from e2b_code_interpreter import Sandbox
import json
import os
import traceback

# Git URLs to process
git_urls = [
    "https://github.com/fireducks-dev/fireducks/",
    "https://github.com/astral-sh/uv"
    # Add more git URLs here
]

# Load sandboxes based on git URLs - each is saved in a sandbox_name.json file
sandboxes = {}

for git_url in git_urls:
    # Extract repo name from git URL
    repo_name = git_url.rstrip('/').split('/')[-1].replace('.git', '')
    sandbox_file = f"sandbox_{repo_name}.json"
    
    try:
        with open(sandbox_file) as f:
            sandbox_info = json.load(f)
        print(f"Sandbox info: {sandbox_info}, type: {type(sandbox_info)}")
        # Connect to the sandbox using the ID from the file
        sandbox_id = sandbox_info.get('sandbox_id')
        print(f"Sandbox ID: {sandbox_id}")
        print(f"Repo name: {repo_name}")
        if sandbox_id:
            # print(f"Connecting to sandbox for {Sandbox.connect(sandbox_id)}")
            sandbox = Sandbox.connect(sandbox_id=sandbox_id, api_key=os.getenv('E2B_API_KEY'))

            sandboxes[repo_name] = sandbox#Sandbox.connect(sandbox_id)
            print(f"Connected to sandbox for {repo_name}: {sandbox_id}")
            # If sandbox has 'fire' in its name, run the test_fireducks code
            if 'fire' in repo_name.lower():
                print(f"Running test_fireducks for {repo_name}...")
                result = sandboxes[repo_name].commands.run("python test_fireducks.py", cwd=f"/sandboxes/{repo_name}")
                print(f"Test output:\n{result.stdout}")
                if result.stderr:
                    print(f"Test errors:\n{result.stderr}")
        else:
            print(f"No sandbox_id found in {sandbox_file}")
            
    except FileNotFoundError:
        print(f"Sandbox file {sandbox_file} not found for {repo_name}")
    except Exception as e:
        print(f"Error loading sandbox for {repo_name}: {e}")
        traceback.print_exc()

print(f"\nLoaded {len(sandboxes)} sandboxes:")
for repo_name in sandboxes.keys():
    print(f"- {repo_name}")

Example of retrieving and running code inside a sandbox

In [ ]:
from e2b_code_interpreter import Sandbox
import json

# Load IDs from the correct file (all_sandbox_ids.json was saved by run_all_pipelines)
with open("all_sandbox_ids.json") as f:
    sandbox_ids = json.load(f)

# Reconnect to all sandboxes using correct syntax
sandboxes = {}
for repo_name, sandbox_id in sandbox_ids.items():
    sandboxes[repo_name] = Sandbox.connect(sandbox_id)

# Now you can use them - example with fireducks
fireducks_sb = sandboxes.get("fireducks")
if fireducks_sb:
    # Test basic Python
    result = fireducks_sb.commands.run("python --version")
    print(f"Python version: {result.stdout}")
    
    # Test fireducks import and basic usage
    fireducks_sb.files.write("test_fireducks.py", """
import fireducks.pandas as pd
import numpy as np

# Create a simple DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': ['a', 'b', 'c', 'd', 'e']
})

print("DataFrame created:")
print(df)

print("\\nDataFrame info:")
print(df.info())

print("\\nDataFrame describe:")
print(df.describe())
""")
    
    result = fireducks_sb.commands.run("python test_fireducks.py")
    print(f"Fireducks test output:\n{result.stdout}")
    if result.stderr:
        print(f"Fireducks test errors:\n{result.stderr}")

# You can also iterate through all sandboxes
print("\nAvailable sandboxes:")
for repo_name, sb in sandboxes.items():
    print(f"- {repo_name}: {sb.sandbox_id}")